In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [ ]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [ ]:
df_2000 = df[df['yearOfLoss']>2000]

In [ ]:
#Zipcode list

list_zipcode = df_2000['reportedZipCode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode = [zipcode.zfill(5) for zipcode in list_zipcode]

In [ ]:
#CensusBG list

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

## BlockGroup shapefile analysis

In [ ]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 400000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
merged_gdf = pd.concat(gdf_list, ignore_index=True)

In [ ]:
merged_gdf.head()

In [ ]:
df_test = merged_gdf['GEOID'].value_counts().reset_index()
df_test.columns = ['GEOID', 'count']

In [ ]:
df_test[df_test['count'] >= 6].count()

In [ ]:
df_test

In [ ]:
list_censusBG_3 = merged_gdf['GEOID'].dropna().drop_duplicates().tolist()

In [ ]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 11-digit BG ids into 12-digits
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_3)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

In [ ]:
len(common_bg)/len(list_censusBG)

In [ ]:
# Assuming you have defined list_censusBG and common_bg
list_BG_not_found = set(list_censusBG) - common_bg
list_BG_not_found = list(list_BG_not_found)

print("CensusBG values not in common:", list_BG_not_found)

In [ ]:
df_BG_not_found = df[['censusBlockGroupFips', 'id']]

In [ ]:
df_cleaned = df_BG_not_found.dropna(subset=['censusBlockGroupFips'])

# Now you can proceed with the remaining operations on df_cleaned
df_cleaned['censusBlockGroupFips'] = [str(int(float(i))) for i in df_cleaned['censusBlockGroupFips']]
df_cleaned['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_cleaned['censusBlockGroupFips']]

In [ ]:
df_cleaned[df_cleaned['censusBlockGroupFips'].isin(list_BG_not_found)].shape[0]

## Zipcode shapefile analysis (until 2000)

In [ ]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 100000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
merged_gdf_zipcode = pd.concat(gdf_list, ignore_index=True)

In [ ]:
merged_gdf_zipcode.shape[0]

In [ ]:
merged_gdf_zipcode.head()

In [ ]:
list_zip = merged_gdf_zipcode['ZIPcode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

list_zip = [zipcode.zfill(5) for zipcode in list_zip]

In [ ]:
list_zip_1 = list(set(list_zip))

In [ ]:
set_zipcode = set(list_zipcode)
set_zipcode_3 = set(list_zip_1)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_3)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(set_zipcode_3)

In [ ]:
len(set_zipcode)

In [ ]:
1-len(common_zipcodes)/len(set_zipcode)

In [ ]:
# Find the difference between set_zipcode and common_zipcodes
zipcodes_not_in_common = set_zipcode - common_zipcodes

# Convert the result back to a list
list_zipcodes_not_in_common = list(zipcodes_not_in_common)

In [ ]:
# Convert the list of zipcodes to a set for faster membership checking
set_zipcodes_not_in_common = set(list_zipcodes_not_in_common)

df_test = df_2000[['reportedZipCode', 'yearOfLoss']]

In [ ]:
df_test['reportedZipCode'] = df_test['reportedZipCode'].dropna().astype(int).astype(str)

In [ ]:
year_list = df_test[df_test['reportedZipCode'].isin(zipcodes_not_in_common)]['yearOfLoss'].unique()

In [ ]:
year_list

## BG Shapefile Analysis..

### BG Shapefile Old

In [2]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 400000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df_old = pd.concat(gdf_list, ignore_index=True)

BG_df_old['year'] = BG_df_old['year'].replace({2012: 2010, 2021: 2020})
BG_df_old = BG_df_old.drop_duplicates(subset=['GEOID', 'year'])

In [4]:
BG_df_old.shape[0]

281234

In [17]:
BG_df_old[BG_df_old['year']==2000].shape[0]

35615

### BF Shapefile New

In [3]:
chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 320000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df_new = pd.concat(gdf_list, ignore_index=True)

In [5]:
BG_df_new.shape[0]

318973

In [16]:
BG_df_new[BG_df_new['year']==2000].shape[0]

73354